<a href="https://colab.research.google.com/github/Anirudh1905/Spam_Classifier/blob/master/Spam_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spam Classifier

In [1]:
import pandas as pd
df=pd.read_csv('spam_sms.csv',encoding='latin',names=['labels','messages'])
df=df.iloc[1:,:]
df.reset_index(inplace=True)
df=df.iloc[:,1:]
df.head()

,labels,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
df.labels.value_counts()

ham     4825
spam     747
Name: labels, dtype: int64

In [3]:
df.isnull().sum()

labels      0
messages    0
dtype: int64

In [4]:
x=list(df['messages'])
y=list(pd.get_dummies(list(df['labels']),drop_first=True)['spam'])

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [9]:
from transformers import DistilBertTokenizerFast
tokenizer=DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [10]:
train_encodings=tokenizer(x_train,truncation=True,padding=True)
test_encodings=tokenizer(x_test,truncation=True,padding=True)

In [11]:
train_encodings[0]

Encoding(num_tokens=238, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [12]:
import tensorflow as tf

train_data = tf.data.Dataset.from_tensor_slices((dict(train_encodings),y_train))
test_data = tf.data.Dataset.from_tensor_slices((dict(test_encodings),y_test))

In [13]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    eval_steps=100
)

In [14]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

trainer = TFTrainer(model=model,args=training_args,train_dataset=train_data,eval_dataset=test_data)

trainer.train()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [15]:
trainer.evaluate(test_data)

{'eval_loss': 0.017173457145690917}

In [16]:
trainer.predict(test_data)

PredictionOutput(predictions=array([[ 3.7032313, -3.2490008],
       [ 2.8945286, -2.541122 ],
       [ 2.2064872, -1.8951063],
       ...,
       [ 3.6187527, -3.148791 ],
       [ 3.6238508, -3.159647 ],
       [ 0.5692362, -0.3395866]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 0], dtype=int32), metrics={'eval_loss': 0.017295631340571813})

In [17]:
y_pred = trainer.predict(test_data)[1]

In [18]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred)

array([[949,   0],
       [  0, 166]])

In [19]:
trainer.save_model('spam_model')